In [5]:
import pandas as pd


Very basic idea here is to create a network of democrats / liberals and segment them for the upcoming primaries (i.e. centrists, leftists, etc.)

In [6]:
import tweepy, sys, os, logging, json
from twitter_config import consumer_key, consumer_secret

In [7]:
logging.basicConfig(filename='twitter_error_log_capt_marv.log',filemode='w', level=logging.ERROR)

First I'm just going to store JSON log files, but eventually I'm going to try to load it into postgres, because I think that makes the most since. I am using the code from https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./

In [8]:
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True,
                 parser=tweepy.parsers.JSONParser()
                )
 
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [9]:
# Here are my search terms - I can change the text string if I want to change my terms
# topic_search = ('aoc OR #aoc OR (alexandria AND ocasio-cortez) OR (alexandria AND ocasio) OR (alexandria AND cortez) OR ocasio-cortez OR ocasiocortez OR (alexandria AND ocasio AND cortez)')
# topic_search = ('(captain AND marvel) OR captainmarvel OR #captainmarvel OR (brie AND larson) OR brielarson OR #brielarson')
topic_search = ('#QAnon OR #Qarmy OR QAnon OR Qarmy OR #WWG1WGA OR WWG1WGA OR #TrustThePlan OR #GreatAwakening')
searchQuery = topic_search

## Download as many search-relevant tweets from the API as possible, while respecting the rate limit.
This code is largely taken from https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./
tweaked it to save json files every 4000 tweets, and to use logging rather than printing.

ALso, adding tweet_mode = extended to every api.search method, and changing langauge as well

In [28]:
maxTweets = 2000000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = 1106243583114252293

# If results only below a specific ID are reqd, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1

tweetCount = 0
tweetFiles = 71
msg =  "Downloading max {0} tweets".format(maxTweets)
logging.error(msg)
tweets = []
while 4000*tweetFiles + tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang='en')
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang='en',
                                        since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang='en',
                                        max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang='en',
                                        max_id=str(max_id - 1),
                                        since_id=sinceId)
                
        new_tweets = new_tweets['statuses']
        if len(new_tweets) == 0:
            msg = 'No tweets found'
            logging.error(msg)
            break
        
        tweets.extend(new_tweets)
        tweetCount += len(new_tweets)
        msg = "Downloaded {0} tweets".format(tweetCount + tweetFiles*4000)
        logging.error(msg)
        max_id = new_tweets[-1]['id']
        
        if tweetCount > 4000:
            with open('tweets'+str(tweetFiles)+'.json', 'w', encoding="utf8") as outfile:  
                json.dump(tweets, outfile)
            msg = "JSON file saved"
            logging.error(msg)
            tweetFiles += 1
            tweetCount = 0
            del tweets[:]
                
    except tweepy.TweepError as e:
        msg = 'Query failed when max_id equaled {0}: {1}'.format(max_id, e)
        logging.error(msg)

if tweetCount > 0:
    with open('tweets'+str(tweetFiles)+'.json', 'w', encoding="utf8") as outfile:  
        json.dump(tweets, outfile)
    msg = "JSON file saved"
    logging.error(msg)
    tweetFiles += 1
    tweetCount = 0

In [29]:
print(max_id)

1106243585958182912


In [17]:
with open('tweets0.json') as json_file:  
    test = json.load(json_file)

In [18]:
len(test)

4071

In [19]:
test[250]

{'created_at': 'Sat Mar 16 21:57:21 +0000 2019',
 'id': 1107038165913620480,
 'id_str': '1107038165913620480',
 'full_text': 'RT @MatzdorfRichard: Nothing to see here….\nJust JP in NZ 5 days before the shooting\nand the "killer" has 14 &amp; fish on his gun…\n\nTheir need…',
 'truncated': False,
 'display_text_range': [0, 143],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'MatzdorfRichard',
    'name': 'Richard Matzdorf',
    'id': 1105885484822220803,
    'id_str': '1105885484822220803',
    'indices': [3, 19]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1046809465,
  'id_str': '1046809465',
  'name': "🇺🇸Tiffany's Epiphany🇺🇸",
  'sc

In [30]:
with open('tweets107.json') as json_file:  
    test_last = json.load(json_file)

In [31]:
test_tweet = test_last[50]


In [32]:
test_tweet['retweeted_status']['full_text']

KeyError: 'retweeted_status'

In [33]:
test_tweet

{'created_at': 'Thu Mar 14 18:01:05 +0000 2019',
 'id': 1106253932110774272,
 'id_str': '1106253932110774272',
 'full_text': "@cspanwj #PatriotsFight #TraitorsJustice #QAnon #WeAreTheNewsNow \nhttps://t.co/fYzXjI3ujh\n\nWhites Were Slaves In North Africa Before Blacks Were Slaves In The New World\n\nBlacks should be paying whites reparations but they have nothing. We'll call it even.",
 'truncated': False,
 'display_text_range': [0, 255],
 'entities': {'hashtags': [{'text': 'PatriotsFight', 'indices': [9, 23]},
   {'text': 'TraitorsJustice', 'indices': [24, 40]},
   {'text': 'QAnon', 'indices': [41, 47]},
   {'text': 'WeAreTheNewsNow', 'indices': [48, 64]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'cspanwj',
    'name': 'Washington Journal',
    'id': 15923226,
    'id_str': '15923226',
    'indices': [0, 8]}],
  'urls': [{'url': 'https://t.co/fYzXjI3ujh',
    'expanded_url': 'https://www.lewrockwell.com/2019/03/paul-craig-roberts/whites-were-slaves-in-north-africa-before-

So now I have the barebones code - I'll create a new notebook where I make functions to select and clean the twitter data I need on just this one dataframe, and then I'll apply it to all of my dataframes and combine them together